# 测试环境

In [4]:
#测试cuda是否可用
import torch
if torch.cuda.is_available():
    print("CUDA is available.")
else:
    print("CUDA is not available.")

CUDA is available.


In [1]:
import torch
device = torch.device("cuda")
#加载数据集
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
X = iris['data']
y = iris['target']
#数据标准化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y, test_size=0.3, random_state=1)

In [2]:
#创建数据集
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
#转为pytorch张量
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train)
train_ds = TensorDataset(X_train,y_train)
torch.manual_seed(1)
batch_size = 2
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [3]:
#设计模型
import torch.nn as nn
import torch.optim as optim
class Model(nn.Module):
    def __init__(self, input_size,hidden_size, output_size):
        super().__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        x = self.layer1(x)
        x = nn.Sigmoid()(x)
        x = self.layer2(x)
        return x
input_size = X_train.shape[1]
hidden_size = 16
output_size = 3
model = Model(input_size, hidden_size, output_size)
#将模型移动到gpu
#model.to(device)

learning_rate = 0.0001
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

e:\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#训练模型
num_epochs = 1000
loss_hist = [0]*num_epochs
accuracy_hist = [0]*num_epochs
for epoch in range(num_epochs):
    for x_batch, y_batch in train_dl:
        y_batch = y_batch.view(-1).long()
        #将数据标签移动到gpu
        #x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        pred = model(x_batch)
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_hist[epoch] += loss.item()*y_batch.size(0)
        is_correct = (torch.argmax(pred, dim=1)) == y_batch.float()
        accuracy_hist[epoch] += is_correct.int().sum().item()
    loss_hist[epoch] /= len(train_dl.dataset)
    accuracy_hist[epoch] /= len(train_dl.dataset)